In [1]:
import numpy as np
import statmapper as stm
from sklearn_tda import MapperComplex
import matplotlib.pyplot as plt
import networkx as nx
from sklearn.cluster import AgglomerativeClustering
from sklearn_tda import clustering

Load data.

In [2]:
X = np.loadtxt("geno_testmat")
XF = np.loadtxt("testfilt")
XF = XF.reshape(467,1)


Compute Mapper with `sklearn_tda`

In [4]:
""" params = {"filters": X[:,2:3], "filter_bnds": np.array([[np.nan,np.nan]]), "colors": X, 
          "resolutions": np.array([24]), "gains": np.array([0.3]), "inp": "point cloud", 
          "clustering": AgglomerativeClustering(n_clusters=None, linkage="single", distance_threshold=.1)} """

""" params = {"filters": XF, "filter_bnds": np.array([[np.nan,np.nan]]), "colors": X, 
          "resolutions": np.array([np.nan]), "gains": np.array([0.50]), "inp": "point cloud", 
          "clustering": AgglomerativeClustering(n_clusters=None, linkage="single", distance_threshold=0.7)} """

params = {"filters": XF, "filter_bnds": np.array([[np.nan,np.nan]]), "colors": X, 
          "resolutions": np.array([np.nan]), "gains": np.array([0.15]), "inp": "point cloud", 
          "clustering": AgglomerativeClustering(n_clusters=None, linkage="single", distance_threshold=382.26814172615633)}

In [5]:
M = MapperComplex(**params).fit(X)

KeyboardInterrupt: 

In [ ]:
MapperComplex?

In [3]:
clustering.estimate_scale(X)

382.26814172615633

In [ ]:
MapperComplex(**params).get_optimal_parameters_for_agglomerative_clustering(X)

Display Mapper with `networkx` library.

In [ ]:
G = stm.mapper2networkx(M)
nx.draw_networkx(G,pos=nx.kamada_kawai_layout(G),
                 node_color=[M.node_info_[node]["colors"][2] for node in G.nodes()])
plt.show()

Define function and topological feature type with which we will compute persistence on the Mapper. In the cell below, the function is defined as the 3rd coordinate of the point cloud and the feature type is downward branch.

In [ ]:
function = XF[:,0]
topo = "downbranch"

Compute downbranches and their representative nodes.

In [ ]:
dgm, bnd = stm.compute_topological_features(M, function, "data", topo)

In [ ]:
print(dgm, bnd)

In [ ]:
G = stm.mapper2networkx(M)
plt.figure(figsize=(40,10))
for idx, bd in enumerate(bnd):
    plt.subplot(1,len(bnd),idx+1)
    nx.draw_networkx(G, pos=nx.kamada_kawai_layout(G), 
            node_color=[1 if node in bd else 0 for node in G.nodes()])
plt.show()

Filter downbranches with significance less than 90% with 100-bootstrap.

In [ ]:
confidence = 0.90
bootstrap  = 100

In [ ]:
sdgm, sbnd = stm.evaluate_significance(dgm, bnd, X, M, function, params, topo, confidence, bootstrap)

In [ ]:
print(sdgm, sbnd)

Sort coordinates with respect to their ability to differentiate the first significant downbranch from the rest of the Mapper nodes.

In [ ]:
features, pv = stm.compute_DE_features(X, M, sbnd[0])

In [ ]:
print(features, pv)